In [1]:
import pickle
dump = pickle.load(open('mem_obj.dump', 'rb'))

In [2]:
len(dump)

689

In [28]:
sum_size = 0
for obj, count_size in zip(dump.keys(), dump.values()):
    if obj == '':
        continue
    wrong_words = {'dalvik', 'boot', 'font', 'Chrome', 'kgsl', 'system', 'lib', 'Audio', 'System', '.dat', '/dev/__properties__', 'dex', 'apk', 'web', 'config', 'cfi', 'binder', '.bss', 'thread', 'bionic'}
    good = True
    for word in wrong_words:
        if word in obj:
            good = False
            break
    if good:
        print(obj)
        print(count_size)
        print('-' * 80)
        sum_size += count_size[1]
print(sum_size)

/dev/ashmem/shared_memory/D5A74A121842A47E7EAF68382AD3794C
[1, 131072]
--------------------------------------------------------------------------------
/dev/ashmem/shared_memory/1D53F3F180B6B1A832373957D4805C13
[1, 28672]
--------------------------------------------------------------------------------
anon_inode:dmabuf
[3, 12288]
--------------------------------------------------------------------------------
[anon:linker_alloc_vector]
[6, 28672]
--------------------------------------------------------------------------------
[anon:linker_alloc]
[69, 323584]
--------------------------------------------------------------------------------
[anon:atexit
[12, 53248]
--------------------------------------------------------------------------------
[anon:linker_alloc_small_objects]
[14, 65536]
--------------------------------------------------------------------------------
/data/data/com.innogames.riseofcultures/no_backup/androidx.work.workdb-shm
[1, 32768]
-----------------------------------

In [31]:
dumps = list()
for i in range(11):
    dumps.append(pickle.load(open(f'{i:03d}.dump', 'rb')))

In [35]:
exists_in_all_dumps = list()
dump_0 = dumps[0]
for key in dump_0.keys():
    in_all = True
    for dump in dumps[1:]:
        if key not in dump:
            in_all = False
            break
    if in_all:
        exists_in_all_dumps.append(key)
print(f'{len(exists_in_all_dumps)} записей сохраняется по всем дампам')

114 записей сохраняется по всем дампам


In [36]:
const_in_all_dumps = list()
for key in exists_in_all_dumps:
    in_all = True
    for dump in dumps[1:]:
        if dump_0[key] != dump[key]:
            in_all = False
            break
    if in_all:
        const_in_all_dumps.append(key)
print(f'{len(const_in_all_dumps)} записей не меняют содержания по всем дампам')

109 записей не меняют содержания по всем дампам


In [37]:
diff_mem = list()
for key in exists_in_all_dumps:
    if key not in const_in_all_dumps:
        diff_mem.append(key)
print('Следующие записи меняются от переговоров к переговорам:')
for key in diff_mem:
    print(key)

Следующие записи меняются от переговоров к переговорам:
('0x7dcf1ea000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
('0x7e502e3000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
('0x7e5042c000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
('0x7e554d5000', 4096, '[vvar]')
('0x7fce384000', 8384512, '[stack]')


In [38]:
'''
Резонно, ведь /dev/ashmem это куча, а стэк не может не меняться.
Хотя на куче меняются очень маленькие аллокации.
Содержимое vvar принадлежит ядру и не может меняться пользовательским кодом.
Стоит сначала исследовать стек.
'''

'\nРезонно, ведь /dev/ashmem это куча, а стэк не может не меняться.\nХотя на куче меняются очень маленькие аллокации.\nСтоит сначала исследовать стек.\n'